In [1]:
from FairFedAvg import *

In [12]:
train(logReg(num_features = NUM_FEATURES, num_classes = 2), 
      option = "FairBatch", optimizer = 'sgd', learning_rate = 0.01, 
      num_rounds = 4, local_epochs = 25, alpha = 0.1)





  0%|          | 0/4 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 1 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 85.929657
| Global Round : 1 | Local Epoch : 0 | [5632/11188 (50%)]	Batch Loss: 47.565742
| Global Round : 1 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 50.182949
| Global Round : 1 | Local Epoch : 1 | [5632/11188 (50%)]	Batch Loss: 38.284527
| Global Round : 1 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 45.353676
| Global Round : 1 | Local Epoch : 2 | [5632/11188 (50%)]	Batch Loss: 53.010597
| Global Round : 1 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 53.220993
| Global Round : 1 | Local Epoch : 3 | [5632/11188 (50%)]	Batch Loss: 52.892109
| Global Round : 1 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 55.345238
| Global Round : 1 | Local Epoch : 4 | [5632/11188 (50%)]	Batch Loss: 38.742771
| Global Round : 1 | Local Epoch : 5 | [0/11188 (0%)]	Batch Loss: 39.663517
| Global Round : 1 | Local Epoch : 5 | [5632/11188 (50%)]	Batch Loss: 57.455147
| Global Round : 1 | Local Epoc





 25%|██▌       | 1/4 [00:22<01:06, 22.06s/it]

Client 0: accuracy loss: 76.14 | fairness loss 0.48 | RD = 0.28 = |97/592-628/1421| 
Client 1: accuracy loss: 40.76 | fairness loss 0.69 | RD = 0.23 = |57/421-298/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 44.98 | Training accuracy: 82.10% | Training RD: 0.2606

 | Global Training Round : 2 |

| Global Round : 2 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 46.341347
| Global Round : 2 | Local Epoch : 0 | [5632/11188 (50%)]	Batch Loss: 55.748508
| Global Round : 2 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 44.703682
| Global Round : 2 | Local Epoch : 1 | [5632/11188 (50%)]	Batch Loss: 45.392906
| Global Round : 2 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 45.710938
| Global Round : 2 | Local Epoch : 2 | [5632/11188 (50%)]	Batch Loss: 48.098320
| Global Round : 2 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 48.584206
| Global Round : 2 | Local Epoch : 3 | [5632/11188 (50%)]	Batch Loss: 50.727989
| Global Round : 2 | Local Epoch : 4 | [0/11188 (0%)]	Batch





 50%|█████     | 2/4 [00:43<00:43, 21.91s/it]

...





 75%|███████▌  | 3/4 [01:04<00:21, 21.63s/it]

Client 0: accuracy loss: 74.31 | fairness loss 0.53 | RD = 0.18 = |121/592-549/1421| 
Client 1: accuracy loss: 39.90 | fairness loss 0.64 | RD = 0.17 = |78/421-289/823| 
 
Avg Training Stats after 3 global rounds:
Training loss: 45.33 | Training accuracy: 82.87% | Training RD: 0.1770

 | Global Training Round : 4 |

| Global Round : 4 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 44.672649
| Global Round : 4 | Local Epoch : 0 | [5632/11188 (50%)]	Batch Loss: 54.129784
| Global Round : 4 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 49.973560
| Global Round : 4 | Local Epoch : 1 | [5632/11188 (50%)]	Batch Loss: 49.685154
| Global Round : 4 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 46.336464
| Global Round : 4 | Local Epoch : 2 | [5632/11188 (50%)]	Batch Loss: 52.265564
| Global Round : 4 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 48.558739
| Global Round : 4 | Local Epoch : 3 | [5632/11188 (50%)]	Batch Loss: 46.403103
| Global Round : 4 | Local Epoch : 4 | [0/11188 (0%)]	Batc





100%|██████████| 4/4 [01:24<00:00, 21.22s/it]

Client 0: accuracy loss: 75.26 | fairness loss 0.52 | RD = 0.04 = |152/592-415/1421| 
Client 1: accuracy loss: 41.83 | fairness loss 0.63 | RD = 0.02 = |100/421-215/823| 
 
Avg Training Stats after 4 global rounds:
Training loss: 45.51 | Training accuracy: 82.39% | Training RD: 0.0320


 
 Results after 5 global rounds of training:
|---- Avg Train Accuracy: 82.39%
|---- Test Accuracy: 83.34%
|---- Test RD: 0.0336

 Total Run Time: 85.2780 sec


In [18]:
train(logReg(num_features=NUM_FEATURES, num_classes=2), 
      option = "FairBatch", optimizer = 'sgd', learning_rate = 0.005, 
      num_rounds = 7, local_epochs = 20, alpha = 0.05)






  0%|          | 0/7 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 1 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 93.492142
| Global Round : 1 | Local Epoch : 0 | [5632/11188 (50%)]	Batch Loss: 46.482395
| Global Round : 1 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 52.444008
| Global Round : 1 | Local Epoch : 1 | [5632/11188 (50%)]	Batch Loss: 35.396774
| Global Round : 1 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 46.407356
| Global Round : 1 | Local Epoch : 2 | [5632/11188 (50%)]	Batch Loss: 50.513691
| Global Round : 1 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 53.005146
| Global Round : 1 | Local Epoch : 3 | [5632/11188 (50%)]	Batch Loss: 48.428680
| Global Round : 1 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 52.011158
| Global Round : 1 | Local Epoch : 4 | [5632/11188 (50%)]	Batch Loss: 37.721016
| Global Round : 1 | Local Epoch : 5 | [0/11188 (0%)]	Batch Loss: 40.968006
| Global Round : 1 | Local Epoch : 5 | [5632/11188 (50%)]	Batch Loss: 52.991093
| Global Round : 1 | Local Epoc






 14%|█▍        | 1/7 [00:17<01:42, 17.13s/it]

Client 0: accuracy loss: 90.37 | fairness loss 0.41 | RD = 0.38 = |113/592-815/1421| 
Client 1: accuracy loss: 45.66 | fairness loss 0.57 | RD = 0.30 = |71/421-386/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 43.95 | Training accuracy: 77.97% | Training RD: 0.3536

 | Global Training Round : 2 |

| Global Round : 2 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 51.878628
| Global Round : 2 | Local Epoch : 0 | [5632/11188 (50%)]	Batch Loss: 43.565895
| Global Round : 2 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 44.484188
| Global Round : 2 | Local Epoch : 1 | [5632/11188 (50%)]	Batch Loss: 40.785686
| Global Round : 2 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 44.987438
| Global Round : 2 | Local Epoch : 2 | [5632/11188 (50%)]	Batch Loss: 46.361164
| Global Round : 2 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 50.174637
| Global Round : 2 | Local Epoch : 3 | [5632/11188 (50%)]	Batch Loss: 43.645393
| Global Round : 2 | Local Epoch : 4 | [0/11188 (0%)]	Batc






 29%|██▊       | 2/7 [00:34<01:26, 17.28s/it]

...






 86%|████████▌ | 6/7 [01:43<00:17, 17.31s/it]

Client 0: accuracy loss: 72.47 | fairness loss 0.50 | RD = 0.12 = |127/592-475/1421| 
Client 1: accuracy loss: 40.08 | fairness loss 0.59 | RD = 0.11 = |81/421-248/823| 
 
Avg Training Stats after 6 global rounds:
Training loss: 43.66 | Training accuracy: 83.23% | Training RD: 0.1169

 | Global Training Round : 7 |

| Global Round : 7 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 43.661907
| Global Round : 7 | Local Epoch : 0 | [5632/11188 (50%)]	Batch Loss: 49.025993
| Global Round : 7 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 45.627163
| Global Round : 7 | Local Epoch : 1 | [5632/11188 (50%)]	Batch Loss: 46.408180
| Global Round : 7 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 46.159023
| Global Round : 7 | Local Epoch : 2 | [5632/11188 (50%)]	Batch Loss: 54.702171
| Global Round : 7 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 48.781593
| Global Round : 7 | Local Epoch : 3 | [5632/11188 (50%)]	Batch Loss: 47.555332
| Global Round : 7 | Local Epoch : 4 | [0/11188 (0%)]	Batc






100%|██████████| 7/7 [02:00<00:00, 17.25s/it]

Client 0: accuracy loss: 73.83 | fairness loss 0.50 | RD = 0.04 = |142/592-393/1421| 
Client 1: accuracy loss: 41.43 | fairness loss 0.59 | RD = 0.03 = |92/421-204/823| 
 
Avg Training Stats after 7 global rounds:
Training loss: 43.66 | Training accuracy: 82.84% | Training RD: 0.0350


 
 Results after 8 global rounds of training:
|---- Avg Train Accuracy: 82.84%
|---- Test Accuracy: 83.47%
|---- Test RD: 0.0330

 Total Run Time: 121.2015 sec
